In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
chung_extracted=pd.read_csv('chung_extracted_camera.csv')
chung_extracted.head()

,identifier,name,optical_zoom,dimension_unit,dimension_width,dimension_height,dimension_depth,weight_unit,weight_value,iso_range,video_resolution,price_currency,megapixels,battery,has_wifi,has_bluetooth,brand
0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],['457'],['g'],[],['4k'],"['$', 'gbp', '']",['24.3'],['li-ion'],[],[],['fujifilm']
1,argos-589ec2ad7ef7121ac4bc2bfc3d0129e8,['canon eos m50'],[],['cm'],['8.8'],['5.9'],['11.6'],['520'],['g'],['25600'],['4k'],"['$', 'gbp', '']",['24.1'],['rechargeable'],[],[],['canon']
2,argos-a3d833ff0feb474a5a0b34f491d61b84,['canon eos m50'],[],['cm'],['8.8'],['5.9'],['11.6'],['520'],['g'],['25600'],['4k'],"['$', 'gbp', '']",['24.1'],['rechargeable'],[],[],['canon']
3,bestbuy-19aeab402763e2946b8a51a4af4667be,['polaroid originals onestep'],[],[],[],[],[],[],[],"['6400', '3200', '1600']",['4k'],['$'],[],[],['yes'],['yes'],['polaroid']
4,bestbuy-1baf9408004e59e409812ba76dd798be,['canon eos m50'],[],[],[],[],[],[],[],"['6400', '3200', '1600']",['4k'],['$'],"['24.1', '24', '24.1']",['lp-e12'],['yes'],['yes'],['canon']


In [3]:
chung_extracted.shape

(120, 17)

In [4]:
bow_product_catalog=pd.read_csv("cameras_catalog_2.csv",sep=';',
                            dtype={'price': 'S10','optical_zoom':'S10','unit':'S10','width':'S10',
 'height':'S10',
 'depth':'S10',
 'unit2':'S10',
 'value':'S10',
'iso_range':'S10','video_resolution':'S10','megapixels':'S10','battery':'S10','has_wifi':'S10','has_bluetooth':'S10'})#actuals that are in the product catalog
cam_gold_standard=pd.read_csv("cameras_gold_standard_final.csv")#matchings
cam_gold_standard.head()


,page_id,catalog_id,label
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0


In [5]:
bow_product_catalog.head()

,id,id_2,name,brand,type,color,itemCondition,availability,priceCurrency,price,...,unit,width,height,depth,unit2,value,model,has_wifi,has_bluetooth,battery
0,walmart-51650ba6d92bdca372e07f63cf51df06,camera_001,Nikon D3400 Digital SLR Camera with 24.2 Megap...,Nikon,DSLR,Black,NewCondition,InStock,USD,569.99,...,Inches,6.5,5.32,11.02,lbs,4.03,D3400,Yes,yes,EN-EL14A r
1,walmart-698daed2260ea72556cb5cb342d17f78,camera_002,Canon EOS Rebel T6 Digital SLR Camera with 18-...,Canon,DSLR,Black,NewCondition,InStock,USD,449.99,...,inches,5.1,4,3.1,lbs,15.52,1159C003,yes,yes,1 x Canon
2,samys-ad91d35d56d16a8d759dc41c692a0b9c,camera_003,Sony Alpha a68 Digital SLR Camera with DT 18-5...,Sony,DSLR,Black,NewCondition,InStock,USD,698,...,in,5.61,4.1,3.26,g,696,a68,no,no,NP-FM500H
3,samys-63325293c96d4d030868ce8ca9fbc7b7,camera_004,Pentax K-1 Mark II Digital SLR Camera Body,Pentax,DSLR,Black,NewCondition,InStock,USD,"1,996.95",...,In,5.4,4.3,3.4,g,925,K-1 Mark II,yes,no,D-LI90 Rec
4,bestbuy-cba5da0c30ab48b7f2303cc9734ae8c7,camera_005,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,Panasonic,DSLR,Black,NewCondition,InStock,USD,449.99,...,inches,4.41,2.65,1.62,pounds,0.71,DC-ZS70K,yes,no,1 x Li-ion


In [6]:
cam_gold_standard.shape

(6000, 3)

In [7]:
cols=list(chung_extracted.columns.values)
i=0
new_col=[]
while(i<len(cols)):
    new_col.append('page_'+cols[i])
    i=i+1
chung_extracted.columns = new_col

In [8]:
chung_extracted.head()

,page_identifier,page_name,page_optical_zoom,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,page_weight_unit,page_weight_value,page_iso_range,page_video_resolution,page_price_currency,page_megapixels,page_battery,page_has_wifi,page_has_bluetooth,page_brand
0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],['457'],['g'],[],['4k'],"['$', 'gbp', '']",['24.3'],['li-ion'],[],[],['fujifilm']
1,argos-589ec2ad7ef7121ac4bc2bfc3d0129e8,['canon eos m50'],[],['cm'],['8.8'],['5.9'],['11.6'],['520'],['g'],['25600'],['4k'],"['$', 'gbp', '']",['24.1'],['rechargeable'],[],[],['canon']
2,argos-a3d833ff0feb474a5a0b34f491d61b84,['canon eos m50'],[],['cm'],['8.8'],['5.9'],['11.6'],['520'],['g'],['25600'],['4k'],"['$', 'gbp', '']",['24.1'],['rechargeable'],[],[],['canon']
3,bestbuy-19aeab402763e2946b8a51a4af4667be,['polaroid originals onestep'],[],[],[],[],[],[],[],"['6400', '3200', '1600']",['4k'],['$'],[],[],['yes'],['yes'],['polaroid']
4,bestbuy-1baf9408004e59e409812ba76dd798be,['canon eos m50'],[],[],[],[],[],[],[],"['6400', '3200', '1600']",['4k'],['$'],"['24.1', '24', '24.1']",['lp-e12'],['yes'],['yes'],['canon']


In [9]:
page_info=pd.merge(cam_gold_standard, chung_extracted, how='left',left_on=['page_id'],right_on=['page_identifier'])
page_info.shape

(6000, 20)

In [10]:
page_info.head()

,page_id,catalog_id,label,page_identifier,page_name,page_optical_zoom,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,page_weight_unit,page_weight_value,page_iso_range,page_video_resolution,page_price_currency,page_megapixels,page_battery,page_has_wifi,page_has_bluetooth,page_brand
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],['457'],['g'],[],['4k'],"['$', 'gbp', '']",['24.3'],['li-ion'],[],[],['fujifilm']
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],['457'],['g'],[],['4k'],"['$', 'gbp', '']",['24.3'],['li-ion'],[],[],['fujifilm']
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],['457'],['g'],[],['4k'],"['$', 'gbp', '']",['24.3'],['li-ion'],[],[],['fujifilm']
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],['457'],['g'],[],['4k'],"['$', 'gbp', '']",['24.3'],['li-ion'],[],[],['fujifilm']
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],['457'],['g'],[],['4k'],"['$', 'gbp', '']",['24.3'],['li-ion'],[],[],['fujifilm']


- Dataframe for product catalog

In [11]:
bow_product_catalog=bow_product_catalog.drop('id',axis=1)
bow_product_catalog.shape

(50, 23)

In [12]:
bow_product_catalog.head() #contains product catalog info

,id_2,name,brand,type,color,itemCondition,availability,priceCurrency,price,megapixels,...,unit,width,height,depth,unit2,value,model,has_wifi,has_bluetooth,battery
0,camera_001,Nikon D3400 Digital SLR Camera with 24.2 Megap...,Nikon,DSLR,Black,NewCondition,InStock,USD,569.99,24.2,...,Inches,6.5,5.32,11.02,lbs,4.03,D3400,Yes,yes,EN-EL14A r
1,camera_002,Canon EOS Rebel T6 Digital SLR Camera with 18-...,Canon,DSLR,Black,NewCondition,InStock,USD,449.99,18,...,inches,5.1,4,3.1,lbs,15.52,1159C003,yes,yes,1 x Canon
2,camera_003,Sony Alpha a68 Digital SLR Camera with DT 18-5...,Sony,DSLR,Black,NewCondition,InStock,USD,698,24.8,...,in,5.61,4.1,3.26,g,696,a68,no,no,NP-FM500H
3,camera_004,Pentax K-1 Mark II Digital SLR Camera Body,Pentax,DSLR,Black,NewCondition,InStock,USD,"1,996.95",36.4,...,In,5.4,4.3,3.4,g,925,K-1 Mark II,yes,no,D-LI90 Rec
4,camera_005,Panasonic - LUMIX DC-ZS70 20.3-Megapixel Digit...,Panasonic,DSLR,Black,NewCondition,InStock,USD,449.99,20.3,...,inches,4.41,2.65,1.62,pounds,0.71,DC-ZS70K,yes,no,1 x Li-ion


In [13]:
all_df=pd.merge(page_info, bow_product_catalog, how='left',left_on=['catalog_id'],right_on=['id_2'])
all_df.shape

(6000, 43)

In [14]:
all_df.head()

,page_id,catalog_id,label,page_identifier,page_name,page_optical_zoom,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,...,unit,width,height,depth,unit2,value,model,has_wifi,has_bluetooth,battery
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],...,Inches,6.5,5.32,11.02,lbs,4.03,D3400,Yes,yes,EN-EL14A r
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],...,inches,5.1,4,3.1,lbs,15.52,1159C003,yes,yes,1 x Canon
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],...,in,5.61,4.1,3.26,g,696,a68,no,no,NP-FM500H
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],...,In,5.4,4.3,3.4,g,925,K-1 Mark II,yes,no,D-LI90 Rec
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],...,inches,4.41,2.65,1.62,pounds,0.71,DC-ZS70K,yes,no,1 x Li-ion


In [15]:
new_col #has the column names that are extracted by chung

['page_identifier',
 'page_name',
 'page_optical_zoom',
 'page_dimension_unit',
 'page_dimension_width',
 'page_dimension_height',
 'page_dimension_depth',
 'page_weight_unit',
 'page_weight_value',
 'page_iso_range',
 'page_video_resolution',
 'page_price_currency',
 'page_megapixels',
 'page_battery',
 'page_has_wifi',
 'page_has_bluetooth',
 'page_brand']

In [16]:
len(new_col) #16 comparisons will be done

17

In [17]:
cat_values=list(bow_product_catalog.columns.values)
cat_values

['id_2',
 'name',
 'brand',
 'type',
 'color',
 'itemCondition',
 'availability',
 'priceCurrency',
 'price',
 'megapixels',
 'optical_zoom',
 'video_resolution',
 'iso_range',
 'unit',
 'width',
 'height',
 'depth',
 'unit2',
 'value',
 'model',
 'has_wifi',
 'has_bluetooth',
 'battery']

In [18]:
all_df=all_df.drop([
 'type',
 'color',
 'itemCondition',
 'availability',
 'price',
 'model'],axis=1)

In [19]:
all_df.head()

,page_id,catalog_id,label,page_identifier,page_name,page_optical_zoom,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,...,iso_range,unit,width,height,depth,unit2,value,has_wifi,has_bluetooth,battery
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],...,100-25600,Inches,6.5,5.32,11.02,lbs,4.03,Yes,yes,EN-EL14A r
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],...,100-6400,inches,5.1,4,3.1,lbs,15.52,yes,yes,1 x Canon
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],...,100?25600,in,5.61,4.1,3.26,g,696,no,no,NP-FM500H
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],...,100-819200,In,5.4,4.3,3.4,g,925,yes,no,D-LI90 Rec
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],[],['cm'],['13.25'],['9.18'],['4.92'],...,80 - 3200,inches,4.41,2.65,1.62,pounds,0.71,yes,no,1 x Li-ion


In [20]:
all_df.tail()

,page_id,catalog_id,label,page_identifier,page_name,page_optical_zoom,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,...,iso_range,unit,width,height,depth,unit2,value,has_wifi,has_bluetooth,battery
5995,wilkinson-c4986b025354d63a40b8e7378d5a0427,camera_046,0,wilkinson-c4986b025354d63a40b8e7378d5a0427,['title ricoh wg-m2'],[],['in'],['2.3'],['1.3'],['3.1'],...,80 - 3200,mm,120,81.7,91.9,g,442,yes,no,Lithium-Io
5996,wilkinson-c4986b025354d63a40b8e7378d5a0427,camera_047,0,wilkinson-c4986b025354d63a40b8e7378d5a0427,['title ricoh wg-m2'],[],['in'],['2.3'],['1.3'],['3.1'],...,80 - 3200,mm,127.5,89,91.7,g,590,no,no,AA battery
5997,wilkinson-c4986b025354d63a40b8e7378d5a0427,camera_048,0,wilkinson-c4986b025354d63a40b8e7378d5a0427,['title ricoh wg-m2'],[],['in'],['2.3'],['1.3'],['3.1'],...,80 - 3200,mm,82,114,84,g,436,no,no,AA x 4
5998,wilkinson-c4986b025354d63a40b8e7378d5a0427,camera_049,0,wilkinson-c4986b025354d63a40b8e7378d5a0427,['title ricoh wg-m2'],[],['in'],['2.3'],['1.3'],['3.1'],...,80 - 6400,mm,130.2,94.3,119.2,g,616,yes,no,Lithium-Io
5999,wilkinson-c4986b025354d63a40b8e7378d5a0427,camera_050,0,wilkinson-c4986b025354d63a40b8e7378d5a0427,['title ricoh wg-m2'],[],['in'],['2.3'],['1.3'],['3.1'],...,NA,inches,1.4,2.2,3.7,pounds,0.4,yes,yes,Olympus Li


In [21]:
cols=all_df.columns.values
cols

array(['page_id', 'catalog_id', 'label', 'page_identifier', 'page_name',
       'page_optical_zoom', 'page_dimension_unit', 'page_dimension_width',
       'page_dimension_height', 'page_dimension_depth', 'page_weight_unit',
       'page_weight_value', 'page_iso_range', 'page_video_resolution',
       'page_price_currency', 'page_megapixels', 'page_battery',
       'page_has_wifi', 'page_has_bluetooth', 'page_brand', 'id_2', 'name',
       'brand', 'priceCurrency', 'megapixels', 'optical_zoom',
       'video_resolution', 'iso_range', 'unit', 'width', 'height', 'depth',
       'unit2', 'value', 'has_wifi', 'has_bluetooth', 'battery'], dtype=object)

- Preprocessing:
    - Step1: Replace [] values with missing values

In [22]:
i=0
replaced_cols=[
       'page_optical_zoom', 'page_dimension_unit', 'page_dimension_width',
       'page_dimension_height', 'page_dimension_depth', 'page_weight_unit',
       'page_weight_value', 'page_iso_range', 'page_video_resolution',
       'page_price_currency', 'page_megapixels', 'page_battery',
       'page_has_wifi', 'page_has_bluetooth', 'page_brand']
while(i<len(replaced_cols)):
    remove = lambda doc: doc.replace("[]",'')
    list_remove=list(all_df[replaced_cols[i]])
    removed_page = [remove(d) for d in list_remove]
    all_df[replaced_cols[i]]=removed_page
    i=i+1

In [23]:
all_df.head()

,page_id,catalog_id,label,page_identifier,page_name,page_optical_zoom,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,...,iso_range,unit,width,height,depth,unit2,value,has_wifi,has_bluetooth,battery
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],,['cm'],['13.25'],['9.18'],['4.92'],...,100-25600,Inches,6.5,5.32,11.02,lbs,4.03,Yes,yes,EN-EL14A r
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],,['cm'],['13.25'],['9.18'],['4.92'],...,100-6400,inches,5.1,4,3.1,lbs,15.52,yes,yes,1 x Canon
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],,['cm'],['13.25'],['9.18'],['4.92'],...,100?25600,in,5.61,4.1,3.26,g,696,no,no,NP-FM500H
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],,['cm'],['13.25'],['9.18'],['4.92'],...,100-819200,In,5.4,4.3,3.4,g,925,yes,no,D-LI90 Rec
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],,['cm'],['13.25'],['9.18'],['4.92'],...,80 - 3200,inches,4.41,2.65,1.62,pounds,0.71,yes,no,1 x Li-ion


In [24]:
all_df.shape

(6000, 37)

- Preprocessing:
    - Step1: Replace [] values with missing values
    - Step2: Find most frequent value for each column in the dataset

In [25]:
most_freq_value=[]
most_freq_time=[]
replaced_cols=[
       'page_optical_zoom', 'page_dimension_unit', 'page_dimension_width',
       'page_dimension_height', 'page_dimension_depth', 'page_weight_unit',
       'page_weight_value', 'page_iso_range', 'page_video_resolution',
       'page_price_currency', 'page_megapixels', 'page_battery',
       'page_has_wifi', 'page_has_bluetooth', 'page_brand','iso_range']
i=0
while(i<len(replaced_cols)):
    #most frequent key
    kk=all_df[replaced_cols[i]].value_counts().keys()[0]
    aa = all_df[replaced_cols[i]].value_counts().get(kk)
    most_freq_value.append(kk)
    most_freq_time.append(aa)
    i=i+1

In [26]:
most_freq_value

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 "['usd', 'eur', 'gbp']",
 '',
 '',
 "['yes']",
 '',
 "['sony']",
 'NA']

In [27]:
most_freq_time

[4400,
 2750,
 2850,
 2850,
 2750,
 3650,
 3650,
 2800,
 2250,
 2000,
 2550,
 2850,
 3150,
 5000,
 750,
 2520]

In [28]:
mis_dataset=pd.DataFrame([most_freq_value,most_freq_time],columns=replaced_cols)
mis_dataset=mis_dataset.transpose()
mis_dataset.columns=['most_freq_value','freq']
mis_dataset['per']=mis_dataset['freq']/6000
mis_dataset.head()

,most_freq_value,freq,per
page_optical_zoom,,4400,0.733333
page_dimension_unit,,2750,0.458333
page_dimension_width,,2850,0.475
page_dimension_height,,2850,0.475
page_dimension_depth,,2750,0.458333


In [29]:
mis_dataset[mis_dataset['per']>0.50]

,most_freq_value,freq,per
page_optical_zoom,,4400,0.733333
page_weight_unit,,3650,0.608333
page_weight_value,,3650,0.608333
page_has_wifi,['yes'],3150,0.525
page_has_bluetooth,,5000,0.833333


- Preprocessing:
    - Step1: Replace [] values with missing values
    - Step2: Find most frequent value for each column in the dataset
    - Step3: Drop columns that have more than %50 percent missingness

Bottom line: There are lots of missing values in the extracted data, therefore drop columns where there are more than %50 percent missingness.
- page_optical_zoom, page_weight_unit, page_weight_value, page_has_bluetooth are the columns to be dropped.
- we will also drop corresponding columns in product catalog, since we will do comparison, in this case we will also drop: 'optical_zoom','unit2', 'value', 'has_bluetooth'
- Addittionally we are dropping iso values since not ever camera has iso values, and video cameras generally do not have iso settings

In [30]:
all_df=all_df.drop(['page_optical_zoom',
 'page_weight_unit',
 'page_weight_value',
 'page_has_bluetooth'], axis=1)
all_df=all_df.drop(['optical_zoom','unit2', 'value', 'has_bluetooth','iso_range','page_iso_range'], axis=1)

In [31]:
all_df.head()

,page_id,catalog_id,label,page_identifier,page_name,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,page_video_resolution,...,brand,priceCurrency,megapixels,video_resolution,unit,width,height,depth,has_wifi,battery
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],['cm'],['13.25'],['9.18'],['4.92'],['4k'],...,Nikon,USD,24.2,Full HD 10,Inches,6.5,5.32,11.02,Yes,EN-EL14A r
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],['cm'],['13.25'],['9.18'],['4.92'],['4k'],...,Canon,USD,18,Full HD 10,inches,5.1,4,3.1,yes,1 x Canon
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],['cm'],['13.25'],['9.18'],['4.92'],['4k'],...,Sony,USD,24.8,1080 HD,in,5.61,4.1,3.26,no,NP-FM500H
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],['cm'],['13.25'],['9.18'],['4.92'],['4k'],...,Pentax,USD,36.4,1080 HD,In,5.4,4.3,3.4,yes,D-LI90 Rec
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,argos-23426c3985d6e24911336f3d8106409b,['buy fujifilm x'],['cm'],['13.25'],['9.18'],['4.92'],['4k'],...,Panasonic,USD,20.3,3840 x 216,inches,4.41,2.65,1.62,yes,1 x Li-ion


In [32]:
all_df.dtypes

page_id                  object
catalog_id               object
label                     int64
page_identifier          object
page_name                object
page_dimension_unit      object
page_dimension_width     object
page_dimension_height    object
page_dimension_depth     object
page_video_resolution    object
page_price_currency      object
page_megapixels          object
page_battery             object
page_has_wifi            object
page_brand               object
id_2                     object
name                     object
brand                    object
priceCurrency            object
megapixels               object
video_resolution         object
unit                     object
width                    object
height                   object
depth                    object
has_wifi                 object
battery                  object
dtype: object

- Preprocessing:
    - Step1: Replace [] values with missing values
    - Step2: Find most frequent value for each column in the dataset
    - Step3: Drop columns that have more than %50 percent missingness
    - Step4: Replace missingness in other colums, therefore find the second most appearing value

In [33]:
all_df.columns.values

array(['page_id', 'catalog_id', 'label', 'page_identifier', 'page_name',
       'page_dimension_unit', 'page_dimension_width',
       'page_dimension_height', 'page_dimension_depth',
       'page_video_resolution', 'page_price_currency', 'page_megapixels',
       'page_battery', 'page_has_wifi', 'page_brand', 'id_2', 'name',
       'brand', 'priceCurrency', 'megapixels', 'video_resolution', 'unit',
       'width', 'height', 'depth', 'has_wifi', 'battery'], dtype=object)

In [34]:
#preprocessing data removing things like [] and x
i=3
coln=list(all_df.columns.values)
fix = lambda doc: doc.lower().translate(None, "[]'-x").translate(None, '"')
while(i<len(coln)):
    list_ca=list(all_df[coln[i]])
    tokenized_page = [fix(d) for d in list_ca]
    all_df[coln[i]]=tokenized_page
    i=i+1

In [35]:
all_df.head()

,page_id,catalog_id,label,page_identifier,page_name,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,page_video_resolution,...,brand,priceCurrency,megapixels,video_resolution,unit,width,height,depth,has_wifi,battery
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,nikon,usd,24.2,full hd 10,inches,6.5,5.32,11.02,yes,enel14a r
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,canon,usd,18,full hd 10,inches,5.1,4,3.1,yes,1 canon
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,sony,usd,24.8,1080 hd,in,5.61,4.1,3.26,no,npfm500h
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,penta,usd,36.4,1080 hd,in,5.4,4.3,3.4,yes,dli90 rec
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,panasonic,usd,20.3,3840 216,inches,4.41,2.65,1.62,yes,1 liion


In [36]:
#find most frequents for extracted features, pages have no missing values
most_freq=[]
l='[\'"\']'
cols=list([
       'page_dimension_unit', 'page_dimension_width',
       'page_dimension_height', 'page_dimension_depth',
       'page_video_resolution', 'page_price_currency', 'page_megapixels',
       'page_battery', 'page_has_wifi', 'page_brand']) #here we are consentrating on 
i=0
while(i<len(cols)):
    kk=all_df[cols[i]].value_counts().keys()[0]
    if(kk=='' or kk==l):
        most_freq.append(all_df[cols[i]].value_counts().keys()[1])
    else:
        most_freq.append(kk)
    i=i+1

In [37]:
most_freq

['mm',
 '2.3',
 '2.22',
 '3.67',
 '4k',
 'usd, eur, gbp',
 '8, 8',
 'liion',
 'yes',
 'sony']

In [38]:
len(most_freq)

10

In [39]:
all_df.head()

,page_id,catalog_id,label,page_identifier,page_name,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,page_video_resolution,...,brand,priceCurrency,megapixels,video_resolution,unit,width,height,depth,has_wifi,battery
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,nikon,usd,24.2,full hd 10,inches,6.5,5.32,11.02,yes,enel14a r
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,canon,usd,18,full hd 10,inches,5.1,4,3.1,yes,1 canon
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,sony,usd,24.8,1080 hd,in,5.61,4.1,3.26,no,npfm500h
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,penta,usd,36.4,1080 hd,in,5.4,4.3,3.4,yes,dli90 rec
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,panasonic,usd,20.3,3840 216,inches,4.41,2.65,1.62,yes,1 liion


In [40]:
cols

['page_dimension_unit',
 'page_dimension_width',
 'page_dimension_height',
 'page_dimension_depth',
 'page_video_resolution',
 'page_price_currency',
 'page_megapixels',
 'page_battery',
 'page_has_wifi',
 'page_brand']

In [41]:
most_freq[1]

'2.3'

In [42]:
all_df[cols[0]].loc[all_df[cols[0]] =='']

150      
151      
152      
153      
154      
155      
156      
157      
158      
159      
160      
161      
162      
163      
164      
165      
166      
167      
168      
169      
170      
171      
172      
173      
174      
175      
176      
177      
178      
179      
       ..
5420     
5421     
5422     
5423     
5424     
5425     
5426     
5427     
5428     
5429     
5430     
5431     
5432     
5433     
5434     
5435     
5436     
5437     
5438     
5439     
5440     
5441     
5442     
5443     
5444     
5445     
5446     
5447     
5448     
5449     
Name: page_dimension_unit, Length: 3750, dtype: object

In [43]:
all_df[cols[0]].loc[all_df[cols[0]] ==''] = most_freq[0]
all_df[cols[1]].loc[all_df[cols[1]] ==''] = most_freq[1]
all_df[cols[2]].loc[all_df[cols[2]] ==''] = most_freq[2]
all_df[cols[3]].loc[all_df[cols[3]] ==''] = most_freq[3]
all_df[cols[4]].loc[all_df[cols[4]] ==''] = most_freq[4]
all_df[cols[5]].loc[all_df[cols[5]] ==''] = most_freq[5]
all_df[cols[6]].loc[all_df[cols[6]] ==''] = most_freq[6]
all_df[cols[7]].loc[all_df[cols[7]] ==''] = most_freq[7]
all_df[cols[8]].loc[all_df[cols[8]] ==''] = most_freq[8]
all_df[cols[9]].loc[all_df[cols[9]] ==''] = most_freq[9]

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [44]:
all_df.tail(1000)

,page_id,catalog_id,label,page_identifier,page_name,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,page_video_resolution,...,brand,priceCurrency,megapixels,video_resolution,unit,width,height,depth,has_wifi,battery
5000,walmart-05fe45f533e5fb65b2d5764ca611fc8f,camera_001,0,walmart05fe45f533e5fb65b2d5764ca611fc8f,eos rebel t6,mm,2.3,2.22,3.67,4k,...,nikon,usd,24.2,full hd 10,inches,6.5,5.32,11.02,yes,enel14a r
5001,walmart-05fe45f533e5fb65b2d5764ca611fc8f,camera_002,1,walmart05fe45f533e5fb65b2d5764ca611fc8f,eos rebel t6,mm,2.3,2.22,3.67,4k,...,canon,usd,18,full hd 10,inches,5.1,4,3.1,yes,1 canon
5002,walmart-05fe45f533e5fb65b2d5764ca611fc8f,camera_003,0,walmart05fe45f533e5fb65b2d5764ca611fc8f,eos rebel t6,mm,2.3,2.22,3.67,4k,...,sony,usd,24.8,1080 hd,in,5.61,4.1,3.26,no,npfm500h
5003,walmart-05fe45f533e5fb65b2d5764ca611fc8f,camera_004,0,walmart05fe45f533e5fb65b2d5764ca611fc8f,eos rebel t6,mm,2.3,2.22,3.67,4k,...,penta,usd,36.4,1080 hd,in,5.4,4.3,3.4,yes,dli90 rec
5004,walmart-05fe45f533e5fb65b2d5764ca611fc8f,camera_005,0,walmart05fe45f533e5fb65b2d5764ca611fc8f,eos rebel t6,mm,2.3,2.22,3.67,4k,...,panasonic,usd,20.3,3840 216,inches,4.41,2.65,1.62,yes,1 liion
5005,walmart-05fe45f533e5fb65b2d5764ca611fc8f,camera_006,0,walmart05fe45f533e5fb65b2d5764ca611fc8f,eos rebel t6,mm,2.3,2.22,3.67,4k,...,olympus,usd,16.1,hd 1080p,inches,9.5,13,3,no,1 lithium
5006,walmart-05fe45f533e5fb65b2d5764ca611fc8f,camera_007,0,walmart05fe45f533e5fb65b2d5764ca611fc8f,eos rebel t6,mm,2.3,2.22,3.67,4k,...,fujifilm,usd,16.3,1080 hd,inches,1.75,4.43,2.54,no,np95 rech
5007,walmart-05fe45f533e5fb65b2d5764ca611fc8f,camera_008,0,walmart05fe45f533e5fb65b2d5764ca611fc8f,eos rebel t6,mm,2.3,2.22,3.67,4k,...,sony,usd,8.2,1920 108,inches,1.2,1.9,3.3,yes,npb1
5008,walmart-05fe45f533e5fb65b2d5764ca611fc8f,camera_009,0,walmart05fe45f533e5fb65b2d5764ca611fc8f,eos rebel t6,mm,2.3,2.22,3.67,4k,...,nikon,usd,20.3,3840 216,inches,2.4,2.4,2.6,yes,enel12 re
5009,walmart-05fe45f533e5fb65b2d5764ca611fc8f,camera_010,0,walmart05fe45f533e5fb65b2d5764ca611fc8f,eos rebel t6,mm,2.3,2.22,3.67,4k,...,gopro,usd,10,3840 216,inches,1.5,1.43,1.5,yes,lithium io


In [45]:
all_df.columns.values

array(['page_id', 'catalog_id', 'label', 'page_identifier', 'page_name',
       'page_dimension_unit', 'page_dimension_width',
       'page_dimension_height', 'page_dimension_depth',
       'page_video_resolution', 'page_price_currency', 'page_megapixels',
       'page_battery', 'page_has_wifi', 'page_brand', 'id_2', 'name',
       'brand', 'priceCurrency', 'megapixels', 'video_resolution', 'unit',
       'width', 'height', 'depth', 'has_wifi', 'battery'], dtype=object)

# 1. Jaccard

Calculate jaccard by treating all values as text

## 1.1. Calculate Jaccard & Find Weights

In [46]:
page_columns=['page_name',
       'page_dimension_unit', 'page_dimension_width',
       'page_dimension_height', 'page_dimension_depth',
       'page_video_resolution', 'page_price_currency', 'page_megapixels',
       'page_battery','page_has_wifi', 'page_brand']
cat_columns=['name', 'unit','width','height','depth','video_resolution','priceCurrency', 'megapixels','battery','has_wifi','brand']
jac_columns=['jac_name','jac_unit','jac_width','jac_height','jac_depth','jac_video_resolution','jac_priceCurrency',
       'jac_megapixels', 'jac_battery','jac_has_wifi', 'jac_brand']

In [47]:
df = pd.DataFrame(columns=jac_columns)
df = df.fillna(0) # with 0s rather than NaNs
df.head()

,jac_name,jac_unit,jac_width,jac_height,jac_depth,jac_video_resolution,jac_priceCurrency,jac_megapixels,jac_battery,jac_has_wifi,jac_brand


In [48]:
# inspired by http://billchambers.me/tutorials/2014/12/22/cosine-similarity-explained-in-python.html
import string
import math
from __future__ import division
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [49]:
i=0
tokenize = lambda doc: doc.lower().translate(None, "[]'-x").split(" ")#replace('[','').replace()
while(i<len(jac_columns)):
    ls=[]
    k=0
    list_page=list(all_df[page_columns[i]])
    list_ca=list(all_df[cat_columns[i]])
    tokenized_page = [tokenize(d) for d in list_page] # tokenized docs
    tokenized_ca = [tokenize(d) for d in list_ca] # tokenized docs
    while(k<all_df.shape[0]):
        ls.append(jaccard_similarity(tokenized_page[k],tokenized_ca[k]))
        k=k+1
    df[jac_columns[i]]=ls
    i=i+1

In [50]:
df.head()

,jac_name,jac_unit,jac_width,jac_height,jac_depth,jac_video_resolution,jac_priceCurrency,jac_megapixels,jac_battery,jac_has_wifi,jac_brand
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0
4,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,1.0,0.0


In [51]:
df['label']=page_info['label']

In [52]:
df.head()

,jac_name,jac_unit,jac_width,jac_height,jac_depth,jac_video_resolution,jac_priceCurrency,jac_megapixels,jac_battery,jac_has_wifi,jac_brand,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0
4,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,1.0,0.0,0


In [53]:
df.describe()

,jac_name,jac_unit,jac_width,jac_height,jac_depth,jac_video_resolution,jac_priceCurrency,jac_megapixels,jac_battery,jac_has_wifi,jac_brand,label
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,0.011390,0.186667,0.013500,0.004333,0.003333,0.025208,0.048167,0.019333,0.022461,0.520000,0.077389,0.019667
std,0.044299,0.389034,0.115412,0.065691,0.057644,0.078069,0.116674,0.109115,0.100754,0.499641,0.267160,0.138864
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,0.500000,1.000000,1.000000,1.000000,1.000000,1.000000,0.333333,1.000000,1.000000,1.000000,1.000000,1.000000


## 1.2. Modeling Unbalanced

In [54]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

def confusion_matrix_report(y_true, y_pred):
    cm, labels = confusion_matrix(y_true, y_pred), unique_labels(y_true, y_pred)
    column_width = max([len(str(x)) for x in labels] + [5])  # 5 is value length
    report = " " * column_width + " " + "{:_^{}}".format("Prediction", column_width * len(labels))+ "\n"
    report += " " * column_width + " ".join(["{:>{}}".format(label, column_width) for label in labels]) + "\n"
    for i, label1 in enumerate(labels):
        report += "{:>{}}".format(label1, column_width) + " ".join(["{:{}d}".format(cm[i, j], column_width) for j in range(len(labels))]) + "\n"
    return report
    
def analytics(y_true, y_pred):#target of the test data #predictions as 0,1 #model (knnclassifier) #data_test
    print("Confusion Matrix:")
    print(confusion_matrix_report(y_true, y_pred))
    print("Accuracy Score:")
    print(accuracy_score(y_true, y_pred))
    print()
    print("Classification Report:")
    print(classification_report(y_true, y_pred))
    #print("AUC Score:")
    #print(AUC(target_test, model, data_test))

### 1.2.1. Decision Tree

In [55]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [56]:
X=df.drop(['label'],axis=1)
y=df['label']

In [57]:
parameters = [{"criterion": ["gini", "entropy"],
              "min_samples_split": [2, 10, 20],
              "max_depth": [None, 5, 10, 15],
              "min_samples_leaf": [1, 5, 10],
              "max_leaf_nodes": [None, 5, 10, 20],
              }]
clf = GridSearchCV(DecisionTreeClassifier(), parameters, cv=5)
clf.fit(X, y)
print(clf.best_params_)

{'min_samples_split': 2, 'max_leaf_nodes': 20, 'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1}


In [58]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

desicion_tree=DecisionTreeClassifier(min_samples_split= 10, max_leaf_nodes= None, criterion= 'gini', max_depth= 10, min_samples_leaf= 1)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

target_prediction = cross_val_predict(desicion_tree, X, y, cv=cv)

In [59]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0 5864    18
    1   32    86

Accuracy Score:
0.991666666667
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5882
          1       0.83      0.73      0.77       118

avg / total       0.99      0.99      0.99      6000



### 1.2.2. Logistic Regression

In [60]:
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV
#logreg = linear_model.LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X, y)
print(clf.best_params_)

{'penalty': 'l1', 'C': 1000}


In [61]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l1', C= 10)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

In [62]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0 5868    14
    1   38    80

Accuracy Score:
0.991333333333
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5882
          1       0.85      0.68      0.75       118

avg / total       0.99      0.99      0.99      6000



### 1.2.3. Nearest Centroid

In [63]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
logreg = NearestCentroid()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

In [64]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0 5529   353
    1    7   111

Accuracy Score:
0.94
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.94      0.97      5882
          1       0.24      0.94      0.38       118

avg / total       0.98      0.94      0.96      6000



### 1.2.4. Random Forest

In [65]:
from sklearn.ensemble import RandomForestClassifier
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = RandomForestClassifier(n_estimators=20)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

In [66]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0 5870    12
    1   33    85

Accuracy Score:
0.9925
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5882
          1       0.88      0.72      0.79       118

avg / total       0.99      0.99      0.99      6000



## 1.3. Modeling Balanced

Since our dataset is very unbalanced, we decided to balance them by removing same rows

In [67]:
df['label'].value_counts() #highly unbalanced

0    5882
1     118
Name: label, dtype: int64

In [68]:
df.shape

(6000, 12)

In [69]:
df2=df.drop_duplicates()
df2.shape

(406, 12)

### 1.3.1. Decision Trees

In [70]:
X=df2.drop(['label'],axis=1)
y=df2['label']

In [71]:
parameters = [{"criterion": ["gini", "entropy"],
              "min_samples_split": [2, 10, 20],
              "max_depth": [None, 5, 10, 15],
              "min_samples_leaf": [1, 5, 10],
              "max_leaf_nodes": [None, 5, 10, 20],
              }]
clf = GridSearchCV(DecisionTreeClassifier(), parameters, cv=5)
clf.fit(X, y)
print(clf.best_params_)

{'min_samples_split': 20, 'max_leaf_nodes': None, 'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1}


In [72]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

desicion_tree=DecisionTreeClassifier(min_samples_split= 2, max_leaf_nodes= None, criterion= 'entropy', max_depth= 5, min_samples_leaf= 1)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

target_prediction = cross_val_predict(desicion_tree, X, y, cv=cv)

In [73]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0  321     7
    1   29    49

Accuracy Score:
0.911330049261
()
Classification Report:
             precision    recall  f1-score   support

          0       0.92      0.98      0.95       328
          1       0.88      0.63      0.73        78

avg / total       0.91      0.91      0.91       406



### 1.3.2. Logistic Regression

In [74]:
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV
#logreg = linear_model.LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X, y)
print(clf.best_params_)

{'penalty': 'l2', 'C': 10}


In [75]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l1', C= 100)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

In [76]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0  315    13
    1   23    55

Accuracy Score:
0.911330049261
()
Classification Report:
             precision    recall  f1-score   support

          0       0.93      0.96      0.95       328
          1       0.81      0.71      0.75        78

avg / total       0.91      0.91      0.91       406



### 1.3.3. Nearest Centroid

In [77]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
logreg = NearestCentroid()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

In [78]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0  219   109
    1    4    74

Accuracy Score:
0.721674876847
()
Classification Report:
             precision    recall  f1-score   support

          0       0.98      0.67      0.79       328
          1       0.40      0.95      0.57        78

avg / total       0.87      0.72      0.75       406



### 1.3.4. Random Forest

In [79]:
from sklearn.ensemble import RandomForestClassifier
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = RandomForestClassifier(n_estimators=20)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

In [80]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0  308    20
    1   28    50

Accuracy Score:
0.881773399015
()
Classification Report:
             precision    recall  f1-score   support

          0       0.92      0.94      0.93       328
          1       0.71      0.64      0.68        78

avg / total       0.88      0.88      0.88       406



# 2. Use Gaussian kernel 

We decided to use the gaussian kernel in order to compare two integer values, rather than treating them as text

In [81]:
df.shape

(6000, 12)

In [82]:
page_columns=['page_name',
       'page_dimension_unit', 'page_dimension_width',
       'page_dimension_height', 'page_dimension_depth',
       'page_video_resolution', 'page_price_currency', 'page_megapixels',
       'page_battery','page_has_wifi', 'page_brand']
cat_columns=['name', 'unit','width','height','depth','video_resolution','priceCurrency', 'megapixels','battery','has_wifi','brand']
jac_columns=['jac_name','jac_unit','jac_width','jac_height','jac_depth','jac_video_resolution','jac_priceCurrency',
       'jac_megapixels', 'jac_battery','jac_has_wifi', 'jac_brand']

In [83]:
df = pd.DataFrame(columns=jac_columns)
df = df.fillna(0) # with 0s rather than NaNs
df.head()

,jac_name,jac_unit,jac_width,jac_height,jac_depth,jac_video_resolution,jac_priceCurrency,jac_megapixels,jac_battery,jac_has_wifi,jac_brand


In [84]:
# inspired by http://billchambers.me/tutorials/2014/12/22/cosine-similarity-explained-in-python.html
import string
import math
from __future__ import division
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [85]:
all_df.head()

,page_id,catalog_id,label,page_identifier,page_name,page_dimension_unit,page_dimension_width,page_dimension_height,page_dimension_depth,page_video_resolution,...,brand,priceCurrency,megapixels,video_resolution,unit,width,height,depth,has_wifi,battery
0,argos-23426c3985d6e24911336f3d8106409b,camera_001,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,nikon,usd,24.2,full hd 10,inches,6.5,5.32,11.02,yes,enel14a r
1,argos-23426c3985d6e24911336f3d8106409b,camera_002,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,canon,usd,18,full hd 10,inches,5.1,4,3.1,yes,1 canon
2,argos-23426c3985d6e24911336f3d8106409b,camera_003,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,sony,usd,24.8,1080 hd,in,5.61,4.1,3.26,no,npfm500h
3,argos-23426c3985d6e24911336f3d8106409b,camera_004,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,penta,usd,36.4,1080 hd,in,5.4,4.3,3.4,yes,dli90 rec
4,argos-23426c3985d6e24911336f3d8106409b,camera_005,0,argos23426c3985d6e24911336f3d8106409b,buy fujifilm,cm,13.25,9.18,4.92,4k,...,panasonic,usd,20.3,3840 216,inches,4.41,2.65,1.62,yes,1 liion


Following columns can be treated as numerical and then gaussian kernel can be calculated
- page_dimension_width & width   can be treated as integer 
- page_dimension_height & height   can be treated as integer 

In [86]:
all_df.dtypes

page_id                  object
catalog_id               object
label                     int64
page_identifier          object
page_name                object
page_dimension_unit      object
page_dimension_width     object
page_dimension_height    object
page_dimension_depth     object
page_video_resolution    object
page_price_currency      object
page_megapixels          object
page_battery             object
page_has_wifi            object
page_brand               object
id_2                     object
name                     object
brand                    object
priceCurrency            object
megapixels               object
video_resolution         object
unit                     object
width                    object
height                   object
depth                    object
has_wifi                 object
battery                  object
dtype: object

In [87]:
all_df[['page_dimension_width','page_dimension_height','width','height']]=all_df[['page_dimension_width','page_dimension_height','width','height']].apply(pd.to_numeric)

In [88]:
#calculate jaccard
i=0
tokenize = lambda doc: doc.lower().translate(None, "[]'-x").split(" ")#replace('[','').replace()
jac_columns_sub=['jac_name','jac_unit','jac_depth','jac_video_resolution','jac_priceCurrency',
       'jac_megapixels', 'jac_battery','jac_has_wifi', 'jac_brand']
page_columns_sub=['page_name',
       'page_dimension_unit', 'page_dimension_depth',
       'page_video_resolution', 'page_price_currency', 'page_megapixels',
       'page_battery','page_has_wifi', 'page_brand']
cat_columns_sub=['name', 'unit','depth','video_resolution','priceCurrency', 'megapixels','battery','has_wifi','brand']


while(i<len(jac_columns_sub)):
    ls=[]
    k=0
    list_page=list(all_df[page_columns_sub[i]])
    list_ca=list(all_df[cat_columns_sub[i]])
    tokenized_page = [tokenize(d) for d in list_page] # tokenized docs
    tokenized_ca = [tokenize(d) for d in list_ca] # tokenized docs
    while(k<all_df.shape[0]):
        ls.append(jaccard_similarity(tokenized_page[k],tokenized_ca[k]))
        k=k+1
    df[jac_columns_sub[i]]=ls
    i=i+1

In [89]:
df.head()

,jac_name,jac_unit,jac_width,jac_height,jac_depth,jac_video_resolution,jac_priceCurrency,jac_megapixels,jac_battery,jac_has_wifi,jac_brand
0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.000000,1.0,0.0
1,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.000000,1.0,0.0
2,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.000000,1.0,0.0
4,0.1,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.333333,1.0,0.0


In [90]:
#calculate gauissan kernel
i=0
tokenize = lambda doc: doc.lower().translate(None, "[]'-x").split(" ")#replace('[','').replace()
jac_columns_sub=['jac_width','jac_height']
page_columns_sub=['page_dimension_width',
       'page_dimension_height']
cat_columns_sub=['width','height']


while(i<len(jac_columns_sub)):
    ls=[]
    k=0
    list_page=list(all_df[page_columns_sub[i]])
    list_ca=list(all_df[cat_columns_sub[i]])
    while(k<all_df.shape[0]):
        ls.append(round(math.exp(-math.pow(list_page[i]-list_ca[i],2)/2),5))
        k=k+1
    df[jac_columns_sub[i]]=ls
    i=i+1

In [91]:
df.head()

,jac_name,jac_unit,jac_width,jac_height,jac_depth,jac_video_resolution,jac_priceCurrency,jac_megapixels,jac_battery,jac_has_wifi,jac_brand
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0
4,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,1.0,0.0


In [92]:
df['label']=page_info['label']

## 2.2. Modeling Unbalanced

### 2.2.1. Decision Trees

In [93]:
X=df.drop(['label'],axis=1)
y=df['label']
parameters = [{"criterion": ["gini", "entropy"],
              "min_samples_split": [2, 10, 20],
              "max_depth": [None, 5, 10, 15],
              "min_samples_leaf": [1, 5, 10],
              "max_leaf_nodes": [None, 5, 10, 20],
              }]
clf = GridSearchCV(DecisionTreeClassifier(), parameters, cv=5)
clf.fit(X, y)
print(clf.best_params_)

{'min_samples_split': 2, 'max_leaf_nodes': 20, 'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1}


In [94]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

desicion_tree=DecisionTreeClassifier(min_samples_split= 2, max_leaf_nodes=20, criterion= 'gini', max_depth= 15, min_samples_leaf= 1)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

target_prediction = cross_val_predict(desicion_tree, X, y, cv=cv)

In [95]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0 5865    17
    1   29    89

Accuracy Score:
0.992333333333
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5882
          1       0.84      0.75      0.79       118

avg / total       0.99      0.99      0.99      6000



### 2.2.2. Logistic Regression

In [96]:
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV
#logreg = linear_model.LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X, y)
print(clf.best_params_)


{'penalty': 'l1', 'C': 100}


In [ ]:
analytics(y,target_prediction)

In [97]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l1', C= 100)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

### 2.2.3. Nearest Centroid

In [98]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
logreg = NearestCentroid()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0 5529   353
    1    7   111

Accuracy Score:
0.94
()
Classification Report:
             precision    recall  f1-score   support

          0       1.00      0.94      0.97      5882
          1       0.24      0.94      0.38       118

avg / total       0.98      0.94      0.96      6000



### 2.2.4. Random Forrest

In [99]:

from sklearn.ensemble import RandomForestClassifier
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = RandomForestClassifier(n_estimators=20)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

analytics(y,target_prediction)


Confusion Matrix:
      Prediction
         0     1
    0 5873     9
    1   33    85

Accuracy Score:
0.993
()
Classification Report:
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5882
          1       0.90      0.72      0.80       118

avg / total       0.99      0.99      0.99      6000



## 2.3. Modeling Balanced

In [100]:
df2=df.drop_duplicates()
df2.shape

(393, 12)

### 2.3.1. Decision Tree

In [101]:
X=df2.drop(['label'],axis=1)
y=df2['label']
parameters = [{"criterion": ["gini", "entropy"],
              "min_samples_split": [2, 10, 20],
              "max_depth": [None, 5, 10, 15],
              "min_samples_leaf": [1, 5, 10],
              "max_leaf_nodes": [None, 5, 10, 20],
              }]
clf = GridSearchCV(DecisionTreeClassifier(), parameters, cv=5)
clf.fit(X, y)
print(clf.best_params_)

{'min_samples_split': 2, 'max_leaf_nodes': 20, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1}


In [102]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

desicion_tree=DecisionTreeClassifier(min_samples_split= 2, max_leaf_nodes=20, criterion= 'gini', max_depth= 5, min_samples_leaf= 1)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

target_prediction = cross_val_predict(desicion_tree, X, y, cv=cv)

In [103]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0  297    18
    1   27    51

Accuracy Score:
0.885496183206
()
Classification Report:
             precision    recall  f1-score   support

          0       0.92      0.94      0.93       315
          1       0.74      0.65      0.69        78

avg / total       0.88      0.89      0.88       393



### 2.3.2. Logistic Regression

In [104]:
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV
#logreg = linear_model.LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "penalty": ['l2', 'l1']}
clf = GridSearchCV(linear_model.LogisticRegression(), param_grid, cv=5)
clf.fit(X, y)
print(clf.best_params_)


{'penalty': 'l2', 'C': 10}


In [105]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = linear_model.LogisticRegression(penalty= 'l2', C= 10)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

In [106]:
analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0  303    12
    1   28    50

Accuracy Score:
0.898218829517
()
Classification Report:
             precision    recall  f1-score   support

          0       0.92      0.96      0.94       315
          1       0.81      0.64      0.71        78

avg / total       0.89      0.90      0.89       393



### 2.3.3. Nearest Centroid

In [107]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
logreg = NearestCentroid()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

analytics(y,target_prediction)

Confusion Matrix:
      Prediction
         0     1
    0  203   112
    1    6    72

Accuracy Score:
0.699745547074
()
Classification Report:
             precision    recall  f1-score   support

          0       0.97      0.64      0.77       315
          1       0.39      0.92      0.55        78

avg / total       0.86      0.70      0.73       393



### 2.3.4. Random Forrest

In [108]:

from sklearn.ensemble import RandomForestClassifier
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logreg = RandomForestClassifier(n_estimators=20)
target_prediction = cross_val_predict(logreg, X, y, cv=cv)

analytics(y,target_prediction)


Confusion Matrix:
      Prediction
         0     1
    0  292    23
    1   26    52

Accuracy Score:
0.875318066158
()
Classification Report:
             precision    recall  f1-score   support

          0       0.92      0.93      0.92       315
          1       0.69      0.67      0.68        78

avg / total       0.87      0.88      0.87       393

